In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D,Lambda, Dropout, InputLayer, Input
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

In [ ]:
# Create a list with the filepaths for training and testing
train_img = '../input/petfinder-pawpularity-score/train'

test_img = '../input/petfinder-pawpularity-score/test'

train = pd.read_csv(r'../input/petfinder-pawpularity-score/train.csv')

test = pd.read_csv(r'../input/petfinder-pawpularity-score/test.csv')

sample_submission = pd.read_csv(r'../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train['Id'] = train_img+'/'+train['Id']+'.jpg'
test['Id'] = test_img + '/' +test['Id'] +'.jpg'

In [ ]:
print(f'Number of pictures in the training dataset: {train.shape[0]}\n')
print(f'Number of pictures in the testing dataset: {test.shape[0]}\n')

In [ ]:
# Display some pictures of the dataset
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train.Id[i]))
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
train_img_path = Path('../input/petfinder-pawpularity-score/train')

test_img_path = Path('../input/petfinder-pawpularity-score/test')

In [ ]:
#Function to load and convert images to array

def images_to_array(data_dir,df,image_size):
    image_names = df['Id']
    image_target = df['Pawpularity']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_pixels = load_img(img_name,target_size=image_size)
        X[i] = img_pixels
        y[i] = image_target[i]
        
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y  

In [ ]:
#Selecting image size according to pretrained models
img_size = (299,299,3)
X, y = images_to_array(train_img_path,train,img_size)

In [ ]:
def get_features(model_name, data_preprocessor,weight, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    
    base_model = model_name(weights=weight,
                            include_top=False,
                            input_shape=input_size)(preprocessor)
    
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=128, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    
    return feature_maps

In [ ]:
#Extracting features using Xception
Xception_preprocessor = preprocess_input
Xception_features = get_features(Xception,
                                  Xception_preprocessor,
                                 '../input/keras-pretrained-models/Xception_NoTop_ImageNet.h5',
                                  img_size, X)

In [ ]:
#Callbacks
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
#Building Model
model = Sequential()
model.add(InputLayer(Xception_features.shape[1:]))
model.add(Dropout(0.3))
model.add(Dense(1,activation='linear'))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse'])
model.summary()

# Training the CNN on the Train features and evaluating it on the val data
history = model.fit(Xception_features,y,validation_split=0.20,callbacks=my_callback, epochs = 50, batch_size=128)

In [ ]:
def images_to_array(data_dir,df,image_size):
    image_names = df['Id']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    
    
    for i in range(data_size):
        img_name = image_names[i]
        img_pixels = load_img(img_name,target_size=image_size)
        X[i] = img_pixels
        
        
    ind = np.random.permutation(data_size)
    X = X[ind]
    
    print('Ouptut Data Size: ', X.shape)
    return X

In [ ]:
X = images_to_array(test_img_path,test,img_size)

In [ ]:
#Extracting features using Xception
Xception_preprocessor = preprocess_input
Xception_test_features = get_features(Xception,
                                  Xception_preprocessor,
                                 '../input/keras-pretrained-models/Xception_NoTop_ImageNet.h5',
                                  img_size, X)

In [ ]:
pred_net = model.predict(Xception_test_features)

In [ ]:
sample_submission['Pawpularity' ] = pred_net
sample_submission.to_csv('submission.csv',index=False)